In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from R125_R32_PR3 import configuration

# Load data from csv
data = pd.read_csv('R32R125new2.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(1669700)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.2688)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.7312)
    
    m.fs.properties.PR_kappa_A['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_A['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_A['R32', 'R125'].fix(3.5)
    m.fs.properties.PR_kappa_A['R125', 'R32'].fix(-3.5)
    m.fs.properties.PR_kappa_B['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'R125'].fix(-3.5)
    m.fs.properties.PR_kappa_B['R125', 'R32'].fix(3.5)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
#     m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
#     m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].unfix()
    m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].unfix()
    
#     m.fs.kappas = Constraint(expr=m.fs.properties.PR_kappa['R32', 'R125'] == 
#                     m.fs.properties.PR_kappa['R125', 'R32'])
#     m.fs.properties.PR_kappa['R32', 'R125'] = Expression(expr=value(m.fs.properties.PR_kappa['R125', 'R32']))
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['R32', 'R125'].setlb(-10)
    m.fs.properties.PR_kappa_A['R32', 'R125'].setub(10)

    m.fs.properties.PR_kappa_A['R125', 'R32'].setlb(-10)
    m.fs.properties.PR_kappa_A['R125', 'R32'].setub(10)

    m.fs.properties.PR_kappa_B['R32', 'R125'].setlb(-10)
    m.fs.properties.PR_kappa_B['R32', 'R125'].setub(10)

    m.fs.properties.PR_kappa_B['R125', 'R32'].setlb(-10)
    m.fs.properties.PR_kappa_B['R125', 'R32'].setub(10)
    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 1669700, "x_R32":0.7312, 
             "x_R125":0.2688, "y_R125":0.2501, "y_R32":0.7499}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

# Check that degrees of freedom is 0
# assert degrees_of_freedom(m) == 0

2021-08-21 15:51:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 0


In [7]:
variable_name = ["fs.properties.PR_kappa_A['R32', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'R32']",
                 "fs.properties.PR_kappa_B['R32', 'R125']",
                 "fs.properties.PR_kappa_B['R125', 'R32']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-9

# def SSE(m, data):
#     expr = (
#             (float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2
# #             + (float(data["x_R125"]) -
# #              m.fs.state_block.mole_frac_phase_comp["Liq", "R125"])**2
#            )
    
#     return expr*1e4


In [9]:
pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

obj_value, parameters = pest.theta_est()

2021-08-21 15:51:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-21 15:51:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-21 15:51:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-21 15:51:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-21 15:52:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-21 15:52:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-21 15:52:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-21 15:52:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-08-21 15:52:09 [INFO] idaes

  27  1.9253182e+02 1.06e+00 8.51e+05  -1.0 4.88e+06    -  3.23e-03 8.03e-05f  8
  28  1.9252045e+02 1.06e+00 1.69e+06  -1.0 4.50e+06    -  9.49e-03 1.43e-05f  6
  29  1.8983894e+02 9.93e-01 1.51e+06  -1.0 4.64e+06    -  5.10e-03 7.11e-03f  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  1.9049809e+02 9.91e-01 1.57e+06  -1.0 4.80e+06    -  3.79e-03 2.04e-03f  4
  31  1.8192193e+02 9.90e-01 2.41e+06  -1.0 5.95e+06    -  1.91e-04 7.25e-03f  1
  32  1.7005851e+02 1.00e+00 3.12e+06  -1.0 5.40e+06    -  6.80e-03 1.31e-02f  1
  33  1.7211340e+02 1.00e+00 3.19e+06  -1.0 3.75e+07    -  1.22e-04 6.30e-04h  1
  34  1.7695692e+02 1.00e+00 5.06e+06  -1.0 4.75e+07    -  5.18e-03 1.18e-03h  1
  35r 1.7695692e+02 1.00e+00 1.00e+03  -0.0 0.00e+00    -  0.00e+00 2.59e-07R  6
  36r 1.7695693e+02 9.24e-01 9.98e+02  -0.0 2.02e+03    -  2.57e-03 9.90e-04f  1
  37r 1.7695708e+02 9.27e-01 9.95e+02  -0.0 1.29e+03    -  7.68e-03 3.30e-03f  1
  38r 1.7695744e+02 1.67e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 120  1.0300990e+02 2.13e-01 5.36e+05  -2.5 3.13e+04   3.0 6.69e-06 1.44e-04h  1
 121r 1.0300990e+02 2.13e-01 9.99e+02  -0.7 0.00e+00   2.5 0.00e+00 2.84e-08R  2
 122r 1.0300962e+02 1.30e-01 9.97e+02  -0.7 1.85e+04    -  3.60e-03 1.01e-03f  1
 123  1.0349361e+02 1.20e-01 3.07e+01  -2.5 2.60e+05    -  2.23e-02 6.54e-02h  1
 124  1.0349420e+02 1.20e-01 3.07e+01  -2.5 3.79e+05    -  9.51e-02 1.43e-03h  1
 125  1.0538165e+02 9.53e-02 1.57e+02  -2.5 2.63e+05    -  2.55e-02 1.97e-01h  1
 126  1.1082289e+02 2.71e+00 4.81e+02  -2.5 1.09e+05    -  3.92e-03 4.37e-01H  1
 127  1.1130693e+02 2.48e+00 4.29e+02  -2.5 1.30e+05    -  4.35e-01 8.58e-02h  1
 128  1.1376133e+02 1.69e+00 2.07e+02  -2.5 9.82e+04    -  3.88e-01 3.15e-01h  1
 129  1.1376394e+02 1.69e+00 2.06e+02  -2.5 1.53e+05    -  8.55e-03 3.57e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 130  1.1414251e+02 1.62e+00

 213  9.1488112e+01 1.71e-06 3.76e-03  -2.5 2.01e+01    -  1.00e+00 1.00e+00h  1
 214  9.1448376e+01 1.61e-05 9.88e+01  -3.8 2.11e+03    -  3.58e-01 1.00e+00f  1
 215  9.1377065e+01 1.24e-04 1.13e+02  -3.8 3.32e+03    -  7.96e-01 1.00e+00f  1
 216  9.1376123e+01 5.26e-04 1.12e+02  -3.8 8.29e+00  -0.1 4.82e-01 1.00e+00h  1
 217  9.1376105e+01 3.85e-05 1.88e+01  -3.8 1.15e-01  -0.6 1.00e+00 1.00e+00h  1
 218  9.1376075e+01 1.54e-05 5.58e+00  -3.8 1.96e-01  -1.1 1.00e+00 1.00e+00h  1
 219  9.1376058e+01 3.07e-06 9.06e-01  -3.8 1.16e-01  -1.5 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 220  9.1376056e+01 4.93e-08 1.97e-02  -3.8 9.90e-03  -2.0 1.00e+00 1.00e+00h  1
 221  9.1376055e+01 2.84e-10 3.26e-05  -3.8 8.54e-03  -2.5 1.00e+00 1.00e+00h  1
 222  9.1376048e+01 1.84e-09 2.91e-04  -5.7 3.35e-02  -3.0 1.00e+00 1.00e+00h  1
 223  9.1376028e+01 1.67e-08 3.68e-05  -5.7 1.01e-01  -3.4 1.00e+00 1.00e+00h  1
 224  9.1375968e+01 1.55e-07

 306  7.9469544e+01 1.81e-05 2.93e+01  -5.7 1.91e+04    -  1.55e-05 3.97e-04h  8
 307  7.7807182e+01 9.81e-01 1.62e+03  -5.7 7.73e+04    -  2.86e-01 1.00e+00f  1
 308  7.9290185e+01 5.39e-01 9.18e+04  -5.7 1.70e+05    -  3.07e-03 4.50e-01h  1
 309  7.9290368e+01 5.39e-01 9.18e+04  -5.7 3.00e+05    -  7.70e-05 1.18e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 310  7.9291518e+01 5.39e-01 9.18e+04  -5.7 2.90e+05    -  5.38e-10 7.69e-05h  1
 311r 7.9291518e+01 5.39e-01 9.99e+02  -0.3 0.00e+00    -  0.00e+00 1.57e-08R  2
 312r 7.9291518e+01 5.39e-01 9.99e+02  -0.3 4.93e+04    -  8.82e-13 2.14e-05f  1
 313r 7.9292970e+01 3.97e-01 9.87e+02  -0.3 4.90e+04    -  2.04e-02 1.19e-02f  1
 314  7.9193634e+01 3.94e-01 4.33e+02  -5.7 2.11e+05    -  9.29e-03 7.77e-03f  1
 315  7.9090918e+01 2.44e-01 3.32e+02  -5.7 3.74e+04    -  2.33e-01 3.81e-01f  1
 316  7.9211319e+01 2.28e-01 2.57e+03  -5.7 5.10e+04    -  1.73e-07 6.42e-02h  1
 317r 7.9211319e+01 2.28e-01

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 77.662206

The values for the parameters are as follows:
fs.properties.PR_kappa_A[R32,R125] = 0.3925570353260235
fs.properties.PR_kappa_A[R125,R32] = -0.3754372951441287
fs.properties.PR_kappa_B[R32,R125] = -0.3891173728723683
fs.properties.PR_kappa_B[R125,R32] = 0.3892398216526019


In [11]:
pest.ef_instance.display()

Model _Q_opt

  Variables:
    None

  Objectives:
    EF_Obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 77.66220630552104

  Constraints:
    _C_EF_ : Size=124
        Key                       : Lower : Body : Upper
         ('ROOT', 0, 'Scenario1') :   0.0 :  0.0 :   0.0
        ('ROOT', 0, 'Scenario10') :   0.0 :  0.0 :   0.0
        ('ROOT', 0, 'Scenario11') :   0.0 :  0.0 :   0.0
        ('ROOT', 0, 'Scenario12') :   0.0 :  0.0 :   0.0
        ('ROOT', 0, 'Scenario13') :   0.0 :  0.0 :   0.0
        ('ROOT', 0, 'Scenario14') :   0.0 :  0.0 :   0.0
        ('ROOT', 0, 'Scenario15') :   0.0 :  0.0 :   0.0
        ('ROOT', 0, 'Scenario16') :   0.0 :  0.0 :   0.0
        ('ROOT', 0, 'Scenario17') :   0.0 :  0.0 :   0.0
        ('ROOT', 0, 'Scenario18') :   0.0 :  0.0 :   0.0
        ('ROOT', 0, 'Scenario19') :   0.0 :  0.0 :   0.0
         ('ROOT', 0, 'Scenario2') :   0.0 :  0.0 :   0.0
        ('ROOT', 0, 'Scenario20') :   0.0 :  0.0 :   

                    entr_mol_form_vap_comp_ref : Vapor phase molar entropy of formation @ Tref
                        Size=1, Index=None, Units=kg*m**2/K/mol/s**2
                        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
                        None :  None : -269.0 :  None :  True :  True :  Reals
                    pressure_sat_comp_coeff_A : Antoine A coefficient for calculating Psat
                        Size=1, Index=None
                        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
                        None :  None : 4.6932 :  None :  True :  True :  Reals
                    pressure_sat_comp_coeff_B : Antoine B coefficient for calculating Psat
                        Size=1, Index=None, Units=K
                        Key  : Lower : Value       : Upper : Fixed : Stale : Domain
                        None :  None : 1033.304797 :  None :  True :  True :  Reals
                    pressure_sat_comp_coeff_C : Antoine C coefficient for c

                    Key                    : Lower : Body : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 :  0.0 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 :  0.0 :   0.0
                eq_mole_frac_tbub : Size=1
                    Key            : Lower : Body : Upper
                    ('Vap', 'Liq') :   1.0 :  1.0 :   1.0
                _teq_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body : Upper
                    None :   0.0 :  0.0 :   0.0
                eq_temperature_dew : Size=2
                    Key                    : Lower : Body : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 :  0.0 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 :  0.0 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Lower : Body               : Upper
                    ('Vap', 'Liq') :   1.0 : 0.9999999999999999 :   1.0
                equilibrium_constraint : Size=2
                    K

                    ('Vap', 'Liq', 'R125') :   0.0 : -5.329070518200751e-15 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : -3.552713678800501e-15 :   0.0
                eq_mole_frac_tbub : Size=1
                    Key            : Lower : Body : Upper
                    ('Vap', 'Liq') :   1.0 :  1.0 :   1.0
                _teq_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body                   : Upper
                    None :   0.0 : -5.684341886080802e-14 :   0.0
                eq_temperature_dew : Size=2
                    Key                    : Lower : Body                   : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 : -7.105427357601002e-15 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : 1.7763568394002505e-15 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Lower : Body : Upper
                    ('Vap', 'Liq') :   1.0 :  1.0 :   1.0
                equilibrium_constrain

                        Key  : Lower : Value     : Upper : Fixed : Stale : Domain
                        None :  None : 1017.2097 :  None :  True :  True :  Reals
                    pressure_sat_comp_coeff_C : Antoine C coefficient for calculating Psat
                        Size=1, Index=None, Units=K
                        Key  : Lower : Value      : Upper : Fixed : Stale : Domain
                        None :  None : 8.3042e-05 :  None :  True :  True :  Reals
                
                  Objectives:
                    None
                
                  Constraints:
                    None
            Block Scenario3.fs.state_block
            
              Variables:
                flow_mol :  Total molar flowrate
                    Size=1, Index=None, Units=mol/s
                    Key  : Lower : Value : Upper : Fixed : Stale : Domain
                    None :     0 :     1 :  1000 :  True :  True : NonNegativeReals
                mole_frac_comp : Mixture m

...

        
          Constraints:
            None
        
          Blocks:
            Block Scenario28.fs.properties
            
              Variables:
                PR_kappa_A : PRA binary interaction parameters
                    Size=4, Index=Scenario28.fs.properties.PR_kappa_A_index
                    Key              : Lower : Value               : Upper : Fixed : Stale : Domain
                    ('R125', 'R125') :  None :                   0 :  None :  True :  True :  Reals
                     ('R125', 'R32') :   -10 : -0.3754372951441287 :    10 : False : False :  Reals
                     ('R32', 'R125') :   -10 :  0.3925570353260235 :    10 : False : False :  Reals
                      ('R32', 'R32') :  None :                   0 :  None :  True :  True :  Reals
                PR_kappa_B : PRB binary interaction parameters
                    Size=4, Index=Scenario28.fs.properties.PR_kappa_B_index
                    Key              : Lower : Value             

                     ('Vap', 'Liq', 'R32') :     0 : 0.6915218788050133 :  None : False : False :  Reals
                temperature_dew : Dew point temperature
                    Size=1, Index=Scenario28.fs.properties._pe_pairs, Units=K
                    Key            : Lower : Value              : Upper : Fixed : Stale : Domain
                    ('Vap', 'Liq') :    10 : 292.43640538428923 :  1000 : False : False :  Reals
                _mole_frac_tdew : Liquid mole fractions at dew temperature
                    Size=2, Index=Scenario28.fs.state_block._mole_frac_tdew_index
                    Key                    : Lower : Value              : Upper : Fixed : Stale : Domain
                    ('Vap', 'Liq', 'R125') :     0 : 0.3781251185789316 :  None : False : False :  Reals
                     ('Vap', 'Liq', 'R32') :     0 : 0.6218748814210684 :  None : False : False :  Reals
            
              Objectives:
                None
            
              Constrai

                    ('Vap', 'R125') :     0 : 0.3082 :  None :  True :  True :  Reals
                     ('Vap', 'R32') :     0 : 0.6918 :  None : False : False :  Reals
                phase_frac : Phase fractions
                    Size=2, Index=Scenario29.fs.properties.phase_list
                    Key : Lower : Value                 : Upper : Fixed : Stale : Domain
                    Liq :     0 : 0.0005279791438583455 :  None : False : False :  Reals
                    Vap :     0 :    0.9994720208561417 :  None : False : False :  Reals
                _teq : Temperature for calculating phase equilibrium
                    Size=1, Index=Scenario29.fs.properties._pe_pairs, Units=K
                    Key            : Lower : Value             : Upper : Fixed : Stale : Domain
                    ('Vap', 'Liq') :  None : 298.3363645482321 :  None : False : False :  Reals
                _t1_Vap_Liq : Intermediate temperature for calculating Teq
                    Size=1, Inde

                    Key            : Lower : Body : Upper
                    ('Vap', 'Liq') :   1.0 :  1.0 :   1.0
                _teq_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body : Upper
                    None :   0.0 :  0.0 :   0.0
                eq_temperature_dew : Size=2
                    Key                    : Lower : Body                    : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 : -1.7763568394002505e-15 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 :  -3.552713678800501e-15 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Lower : Body : Upper
                    ('Vap', 'Liq') :   1.0 :  1.0 :   1.0
                equilibrium_constraint : Size=2
                    Key                    : Lower : Body                   : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 : 1.3322676295501878e-15 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : 1.3322676295

                    ('Vap', 'Liq', 'R125') :     0 : 0.20881709096457257 :  None : False : False :  Reals
                     ('Vap', 'Liq', 'R32') :     0 :  0.7911829090354274 :  None : False : False :  Reals
            
              Objectives:
                None
            
              Constraints:
                total_flow_balance : Size=1
                    Key  : Lower : Body                  : Upper
                    None :   0.0 : 2.220446049250313e-16 :   0.0
                component_flow_balances : Size=2
                    Key  : Lower : Body                    : Upper
                    R125 :   0.0 : -2.7755575615628914e-17 :   0.0
                     R32 :   0.0 : -1.1102230246251565e-16 :   0.0
                sum_mole_frac : Size=1
                    Key  : Lower : Body : Upper
                    None :   0.0 :  0.0 :   0.0
                phase_fraction_constraint : Size=2
                    Key : Lower : Body : Upper
                    Liq :   0.0

In [12]:
print(large_residuals_set(pest.ef_instance))

ComponentSet([])
